In [12]:
import pyspark 
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf, SparkContext
import os
import re
import numpy as np
import time
from functools import reduce
import pandas as pd

Location_master = 'spark://linux:7077'
Location_master = 'local'
try:
    sparkSession.stop()
except:
    pass
sparkSession = SparkSession \
    .builder \
    .master(Location_master)\
    .appName("Process 2020") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "4g") \
    .config('spark.cores.max','4')\
    .getOrCreate()

In [13]:
data_dir = '../../../Data_process/CrawlBosung/*/'
df_re_crawl = sparkSession.read.csv(data_dir, header=True)
df = sparkSession.read.csv('../../../Data_process/Diemthi2021.csv', header=True)

df_re_crawl = df_re_crawl.drop('_c0')
df_re_crawl.show(5)
df.show(5)

df_re_crawl.printSchema()
df.printSchema()

+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
|     SBD|Cum_thi|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|KHTN|Lich_su|Dia_ly|GDCD|KHXH|
+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
|18000099|     18| 8.2|    6.5|      6.2|   7.5|   7.75|    5.75|null|   null|  null|null|null|
|18000098|     18| 7.6|    6.0|      6.0|  7.75|    8.0|    6.75|null|   null|  null|null|null|
|18000097|     18| 5.6|    6.5|      4.0|  null|   null|    null|null|   3.25|   6.0|8.25|null|
|18000096|     18| 7.0|    7.0|      5.2|  null|   null|    null|null|   3.75|  8.25| 9.5|null|
|18000095|     18| 6.6|    6.5|      6.0|  null|   null|    null|null|   7.25|   8.0| 9.0|null|
+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
only showing top 5 rows

+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
|     SBD|Cum_t

In [14]:
print(df_re_crawl.count())
df_re_crawl = df_re_crawl.drop_duplicates()
print(df_re_crawl.count())
print(df.count())

37395
37395
908997


In [15]:
data_all = reduce(DataFrame.unionAll, [df_re_crawl, df])
data_all = data_all.drop_duplicates()
print(data_all.count())

945460


### Thay đổi kiểu dữ liệu các trường dữ liệu về đúng dạng của nó

In [16]:
import pyspark.sql.functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType, IntegerType

In [17]:
data_all = data_all.drop('_c0')
data_all = data_all.withColumn('Toan', F.col('Toan').cast(FloatType()))
data_all = data_all.withColumn('Ngu_van', F.col('Ngu_van').cast(FloatType()))
data_all = data_all.withColumn('Ngoai_ngu', F.col('Ngoai_ngu').cast(FloatType()))
data_all = data_all.withColumn('Vat_ly', F.col('Vat_ly').cast(FloatType()))
data_all = data_all.withColumn('Hoa_hoc', F.col('Hoa_hoc').cast(FloatType()))
data_all = data_all.withColumn('Sinh_hoc', F.col('Sinh_hoc').cast(FloatType()))
data_all = data_all.withColumn('Lich_su', F.col('Lich_su').cast(FloatType()))
data_all = data_all.withColumn('Dia_ly', F.col('Dia_ly').cast(FloatType()))
data_all = data_all.withColumn('GDCD', F.col('GDCD').cast(FloatType()))

data_all.printSchema()

root
 |-- SBD: string (nullable = true)
 |-- Cum_thi: string (nullable = true)
 |-- Toan: float (nullable = true)
 |-- Ngu_van: float (nullable = true)
 |-- Ngoai_ngu: float (nullable = true)
 |-- Vat_ly: float (nullable = true)
 |-- Hoa_hoc: float (nullable = true)
 |-- Sinh_hoc: float (nullable = true)
 |-- KHTN: string (nullable = true)
 |-- Lich_su: float (nullable = true)
 |-- Dia_ly: float (nullable = true)
 |-- GDCD: float (nullable = true)
 |-- KHXH: string (nullable = true)



In [ ]:
data_all = data_all.drop_duplicates()
print(data_all.count())

In [9]:
from pyspark.sql.window import Window

data_all = data_all.withColumn('row_num', f.row_number().over(
    Window.partitionBy('SBD').orderBy('SBD')
))

In [8]:
# if(data_bosung.count() == 0):
#     data_dup = data_all.filter('row_num = 2').withColumn('raw', f.substring(f.col('SBD'),0,5))
#     data_dup = data_dup.groupBy('raw').count().toPandas()
#     list_data_miss = []
#     list_dup = list(data_dup['raw'].values)
#     for dup in list_dup:
#         for i in range(0,10):
#             pattern = dup + str(i)
#             if(data_all.filter("SBD like '{0}%'".format(pattern)).count() == 0):
#                 list_data_miss.append(pattern)

#     list_data_miss = pd.DataFrame({'Miss': list_data_miss})
#     list_data_miss.to_csv("../../../Data_process/ListMiss/miss.csv")

In [10]:
print(data_all.count())
data_all = data_all.filter('row_num == 1')
data_all = data_all.drop("row_num")
data_all = data_all.drop_duplicates()
print(data_all.count())
data_all = data_all.orderBy("SBD")
data_all.repartition(1).write.format("com.databricks.spark.csv")\
        .option("header", "true").save("mydata.csv", mode='overwrite')

987768
987768


In [15]:
980876 - 969840

11036

In [16]:
sparkSession.stop()